In [7]:
import os
import sys
sys.path.append('..')

import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
from omegaconf import OmegaConf

from vci.inference import Inference
from vci.utils import get_embedding_cfg, get_dataset_cfg

In [8]:
emb_key = 'X_emb'
job_name = 'simple_reverted_1'
config_file = '/home/rajesh.ilango/Projects/vci/vci_pretrain/conf/defaults.yaml'
iter = 234000
chkp_file = f'/large_storage/ctc/userspace/aadduri/vci/checkpoint/nr_mse_de/exp_nr_mse_de_layers_8_dmodel_512_samples_2048_max_lr_0.00024_op_dim_512-epoch=0-step=234000.ckpt'
# chkp_file = f'/scratch/ctc/ML/vci/checkpoint/pretrain/simple_reverted_1/exp_simple_reverted_1_layers_8_dmodel_512_samples_2048_max_lr_0.00024_op_dim_512-epoch=0-step=13000.ckpt'
input_file = '/large_storage/ctc/datasets/vci/validation/replogle_perturbation.h5ad'

dataset_name = os.path.basename(input_file).split('.')[0]
output_emb_file = f'../outputs/{job_name}/{dataset_name}_emb_iter-{iter}.h5ad'
chkp_file

'/large_storage/ctc/userspace/aadduri/vci/checkpoint/nr_mse_de/exp_nr_mse_de_layers_8_dmodel_512_samples_2048_max_lr_0.00024_op_dim_512-epoch=0-step=234000.ckpt'

In [9]:
conf = OmegaConf.load(config_file)
conf

{'experiment': {'name': 'vci_pretrain_${loss.name}_${model.nhead}_${model.nlayers}', 'local': 'local', 'compiled': False, 'deaware': False, 'profile': {'enable_profiler': False, 'profile_steps': [10, 100], 'max_steps': 110}, 'num_epochs': 16, 'num_nodes': 1, 'num_gpus_per_node': 1, 'port': 12399, 'val_check_interval': 1000, 'limit_val_batches': 100, 'checkpoint': {'path': '/scratch/ctc/ML/vci/checkpoint/pretrain', 'save_top_k': 4, 'monitor': 'trainer/train_loss', 'every_n_train_steps': 1000}}, 'wandb': {'enable': True, 'project': 'vci'}, 'embeddings': {'current': 'esm2-cellxgene', 'esm2-cellxgene': {'all_embeddings': '/large_storage/ctc/ML/data/cell/misc/Homo_sapiens.GRCh38.gene_symbol_to_embedding_ESM2.pt', 'ds_emb_mapping': '/large_storage/ctc/datasets/vci/training/gene_embidx_mapping.torch', 'size': 5120}, 'evo2-scbasecamp': {'all_embeddings': '/large_storage/ctc/projects/vci/scbasecamp/all_species_Evo2.torch', 'ds_emb_mapping': '/large_storage/ctc/projects/vci/scbasecamp/dataset_em

In [10]:
# if not os.path.exists(output_emb_file):
inferer = Inference(conf)
inferer.load_model(chkp_file)
os.makedirs(os.path.dirname(output_emb_file), exist_ok=True)
inferer.encode_adata(input_file, output_emb_file, emb_key=emb_key)

!!! Using Flash Attention !!!


TypeError: create_dataloader() got an unexpected keyword argument 'batch_size'

In [ ]:
adata = sc.read_h5ad(output_emb_file)

# Choose embedding method: set use_pca = True for PCA, False for UMAP
use_pca = True  # Change to False to use UMAP
if use_pca:
    from sklearn.decomposition import PCA
    embedding = PCA(n_components=2).fit_transform(adata.obsm[emb_key])
else:
    import umap
    embedding = umap.UMAP(n_components=2).fit_transform(adata.obsm[emb_key])

# Get the cell type information as a categorical series
cell_types = adata.obs['cell_type'].astype('category')

# Create a color palette based on the number of unique cell types
palette = sns.color_palette("hsv", len(cell_types.cat.categories))
color_dict = dict(zip(cell_types.cat.categories, palette))

# Instead of using .map (which may fail with a MultiIndex), use a list comprehension
colors = [color_dict[ct] for ct in cell_types]

# Plot the embedding
plt.figure(figsize=(8, 6))
plt.scatter(embedding[:, 0], embedding[:, 1], c=colors, s=5, alpha=0.7)
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.title("Embedding ({}) for {} ({})".format("PCA" if use_pca else "UMAP", emb_key, job_name + ' ' + str(iter)))


# Create legend handles for each cell type
handles = [
    mlines.Line2D([], [], color=color_dict[ct], marker='o', linestyle='None', markersize=6, label=ct)
    for ct in cell_types.cat.categories
]
plt.legend(handles=handles, title="Cell Type", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()